## 1.3 Pearson's Correlation Coefficient Comparison
This is to test whether the synthetic data has captured dependencies between variables of the original data.  
We calculate Pearson’s correlation coefficients - **_r_**, between categorical columns within the original and synthetic data.   

To compare the how similar those _r_'s are, 
* We calculate the **MSE** between every pair of _r_'s (for the same pair of columns but one from synthetic one from real data)
* We calculate the **SRA** (Synthetic Ranking Agreement,see explanation below) of _r_'s for each column.


In [1]:
def r_corr_test(df,PTable = False, CoefficientandPtable = False, lower = True ):
    '''Returns a table of Pearson's r correlation coefficients between every pair of columns in the dataframe
    
    Args:
    df: The input dataframe
    PTable: False (default) or True, if True, then the return is a table containing the p(probavility)-value of correlation test.
    CoefficientandPtable: False(default) or True, if true, then the return is a table containing tuples (p-value, r coefficient) from the correlation test.
    lower: True(default) or False. If True, the lower triangle part of the table is filled with the transpose of the upper triangle part rather than leaved with None.
    
    Returns:
    The requested table as specified in the args. If PTable and CoefficientandPtable are all False, then the return table consists of coefficient values only.
    
    '''
    from scipy.stats import pearsonr
    import pandas as pd
    import numpy as np

    df_index = (df.keys()).tolist()
    n = len(df_index)
    ini = [ [ None for y in range( n ) ] 
                 for x in range( n ) ]

    #pearsonr returns two values: the correlation coefficient and significance test probability p
    #so we create two empty dataframes to store them
    coefficient_table = pd.DataFrame(ini,index = df_index,columns = df_index)
    p_table = coefficient_table.copy()
    coe_and_p_table = coefficient_table.copy()

    for i in range(n):
        for j in range(i+1,n):
            name1 = df_index[i]
            name2 = df_index[j]
            obs_1 = df[name1].dropna()
            obs_2 = df[name2].dropna()
            dataframe = pd.DataFrame({name1: obs_1, name2: obs_2})

            values = dataframe.dropna().values
            (coe,p) = pearsonr(values[:,0],values[:,1])
            coefficient_table.loc[name1,name2]=coe
            p_table.loc[name1,name2]=p
            coe_and_p_table.loc[name1,name2]=(coe,p)
    
    if lower:
        #A function that can fill the lower part of the dataframe, because coe_table and p_table has their lower triangles empty
        #But for comparison reasons you may want them to be filled
        def fill_lower(df):
            n = df.values.shape[0]
            for j in range(n):
                for i in range(j+1,n):
                    df.iloc[i,j]=df.iloc[j,i]
            return df
        
        coefficient_table = fill_lower(coefficient_table)
        p_table = fill_lower(p_table)
        coe_and_p_table = fill_lower(coe_and_p_table)
    
    
    if PTable:
        return p_table
    elif CoefficientandPtable:
        return coe_and_p_table
    else:
        return coefficient_table

## What is SRA?
**SRA** is used when we want to test whether a synthetic dataset respects a certain ranking. Suppose we have a list of metrics $ R_{1}, R_{2}, R_{3} ... R_{n} $ calculated from the real data and a list of same metrics $ S_{1}, S_{2}, S_{3} ... S_{n} $ calculated from the synthetic data. Then we define **SRA** as
$$
SRA(R,S) = \frac{1}{n(n-1)} \sum_{i=1}^{n}\sum_{j\neq{i}} Id((S_{i}-S_{j})\times(R_{i}-R_{j})>0)\\
$$
where $ Id $ is the identity function. $ SRA \in [0,1] $, The closer the SRA to $ 1 $, the better the ranking agreement.

In the case of correlation comparison, suppose we have columns _A, B, C, D, E_ , for column _A_, we calculate correlation coefficients $ r_{AB}, r_{AC}, r_{AD}, r_{AE} $ for the real data, and $ r'_{AB}, r'_{AC}, r'_{AD}, r'_{AE} $ for the synthetic data. We hope the ranking of $ r $ and $ r' $ agrees, e.g. if $ r_{AB} > r_{AC} $ then $ r'_{AB} > r'_{AC} $ as well. As a result, our $ R $ is $ r_{AB}, r_{AC}, r_{AD}, r_{AE} $, $ S $ is $ r'_{AB}, r'_{AC}, r'_{AD}, r'_{AE} $, and we can calulate the $ SRA $ for each column.

In [2]:
def SRA(R,S):
    '''Calculate the SRA of lists R and S
    
    Args:
    - R: A list of performance metrics of different predictive models from TSTS
    - S: A list of performance metrics of different predictive models from TRTR, len(S)=len(R)
    
    Returns:
    - SRA: SRA value
    
    '''
    def identity_function(statement):
        v = 0
        if statement:
            v = 1
        return v
            
    k = len(R)
    sum_ = 0
    for i in range(k):
        for j in range(k):
            if i != j:
                if (R[i]-R[j])==0:
                    if (S[i]-S[j])==0:
                        agree = True
                    else:
                        agree = False
                else:
                    agree = (R[i]-R[j])*(S[i]-S[j])>0
                sum_ += identity_function(agree)
    SRA = sum_ / (k*(k-1))
    return SRA

In [3]:
def CorrelationSRA(ori_correlation_df,gen_correlation_df,ColumnWise = False):
    '''Returns the value of SRA for the absolute Pearsons correlation coefficients for each column between \
    all other columns. SRA is between 0 and 1, the closer the SRA is to 1, the more the agreement between the ranking,\
    the more similar the synthetic data and the real data are.
    
    Args:
    ori_correlation_df: the correlation coefficient dataframe for the real data, usually generated from the function\
                        r_corr_test.
    gen_correlation_df: the correlation coefficient dataframe for the synthetic data, usually generated from the function\
                        r_corr_test. 
    ColumnWise: False(default) or True. If True, the return is a Series containing the SRA value for each column and the average.\
                Otherwise, the return is the average of SRA values for all columns
    
    Returns:
    s: It is either a column-wise SRA series or the average SRA values of them, determined by the arg ColumnWise.
    
    '''
    import numpy as np
    import pandas as pd
    
    columns = (ori_correlation_df.keys()).tolist()
    n = len(columns)
    ini = np.ones(n)
    
    for i in range(n):
        ori_values = ori_correlation_df.iloc[i,:].dropna()
        gen_values = gen_correlation_df.iloc[i,:].dropna()
        ini[i] = SRA(abs(ori_values), abs(gen_values))
    
    if ColumnWise:
        s = pd.Series(ini,index = columns)
        s['average'] = sum(ini)/n
    else:
        s = sum(ini)/n
    return s

In [4]:
def MSE(r_table_ori,r_table_gen):
    '''
    Returns the MSE for each position between two dataframes and an average value.
    '''
    import pandas as pd
    import numpy as np
    ori = r_table_ori.fillna(0).values
    gen = r_table_gen.fillna(0).values
    columns = (r_table_gen.keys()).tolist()
    matrix = (ori-gen)**2
    df = pd.DataFrame(matrix, index = columns, columns = columns)
    score = np.sum(matrix)/(len(ori)*(len(ori)-1)) #The diagonal is always zero so we don't count them
    return df, score

# Data Loading

In [5]:
import numpy as np
import pandas as pd
dp_ori_df = pd.read_csv('synthetic data/doppelGANger/dp_ori.csv') #originally ori_features_prism.npy
dp_gen_df = pd.read_csv('synthetic data/doppelGANger/dp_gen.csv') #originally features_600.npy
tgan_ori_df = pd.read_csv('synthetic data/TGAN/tgan_ori.csv') #originally cat_time_10visits_all_noid.csv
tgan_gen_df = pd.read_csv('synthetic data/TGAN/tgan_gen.csv') #originally gen_cat_time_10visits_wl_5000it.npy
ori_df = pd.read_csv('synthetic data/2_no_id/ori_df.csv') #originally cat_time_5abovevisits_all.csv
gen_1_df = pd.read_csv('synthetic data/2_no_id/gen_1_df.csv') #originally gen_cat_time_10visits_wl_5000it_hd10_nl5.npy
gen_2_df = pd.read_csv('synthetic data/2_no_id/gen_2_df.csv') #originally gen_cat_time_10visits_wl_5000it_hd10.npy
gen_3_df = pd.read_csv('synthetic data/2_no_id/gen_3_df.csv') #originally gen_dop_cat_5abovevisits_d2g_e449.npy
gen_4_df = pd.read_csv('synthetic data/2_no_id/gen_4_df.csv') #originally from gen_cat_time_10visits_all_5000it.npy.
dp_0827_df = pd.read_csv('synthetic data/2_no_id/dp_0827_gen.csv')  #originally gen_doptf2_cat_5abovevisits_e200_lstm.csv

synthetic_data_dic = {'DoppelGANger_0814':[dp_ori_df, dp_gen_df],'DoppelGANger_0824':[ori_df,gen_3_df],\
                      'DoppelGANger_0827':[ori_df,dp_0827_df],'tGAN':[tgan_ori_df,tgan_gen_df],\
                      'tGAN 1':[tgan_ori_df,gen_1_df],'tGAN 2':[tgan_ori_df,gen_2_df],\
                     'tGAN 4':[tgan_ori_df,gen_4_df]}
syn_keys = list(synthetic_data_dic.keys())

In [6]:
n = len(syn_keys)
MSE_array = np.zeros(n)
for i in range(n):
    key = syn_keys[i]
    df_ori = synthetic_data_dic[key][0]
    df_gen = synthetic_data_dic[key][1]
    r_table_ori = r_corr_test(df_ori)
    r_table_gen = r_corr_test(df_gen)
    
    #Highlight all r values > 0.5 as yellow, indicating strong correlation
    def color_threshold_yellow(val):
        threshold = 0.5
        if ((val != None) and (abs(val) > threshold)):
            color = 'yellow' 
        else:
            color = 'black'
        return 'color: %s' % color

    display(key+' '+'generated r table',r_table_gen.style.applymap(color_threshold_yellow))
    display(key+' '+'real r table',r_table_ori.style.applymap(color_threshold_yellow))
    sra = CorrelationSRA(r_table_ori,r_table_gen,ColumnWise=True)
    if i==0:
        sra_df = pd.DataFrame(sra,columns = [key])
    else:
        sra_df = pd.concat([sra_df,pd.DataFrame(sra,columns = [key])], axis = 1, sort = False)
    display(key+' '+'SRA',sra)
    MSE_df, MSE_score = MSE(r_table_gen,r_table_ori)
    display(key+' '+'MSE table', MSE_df)
    MSE_array[i] = MSE_score
MSE_series = pd.Series(MSE_array,index = syn_keys)

'DoppelGANger_0814 generated r table'

,dday,weight,height,age,temp
dday,None,0.460590,0.579901,0.385757,0.572924
weight,0.460590,None,0.852360,0.765801,0.725225
height,0.579901,0.852360,None,0.700063,0.951644
age,0.385757,0.765801,0.700063,None,0.561933
temp,0.572924,0.725225,0.951644,0.561933,None


'DoppelGANger_0814 real r table'

,dday,weight,height,age,temp
dday,None,0.547442,0.625742,0.431480,0.624604
weight,0.547442,None,0.904009,0.888397,0.787127
height,0.625742,0.904009,None,0.739106,0.964485
age,0.431480,0.888397,0.739106,None,0.589636
temp,0.624604,0.787127,0.964485,0.589636,None


'DoppelGANger_0814 SRA'

...

,dday,height,weight,temp,vomit_dur,cough_dur,diar_No,diar_Yes,head_No,head_Yes
dday,None,0.176702,0.216054,-0.090951,-0.039075,-0.091015,0.017397,-0.017397,0.069809,-0.069809
height,0.176702,None,0.873474,-0.143156,-0.026711,-0.059380,0.053020,-0.053020,-0.108343,0.108343
weight,0.216054,0.873474,None,-0.122154,-0.018737,-0.041915,0.015691,-0.015691,-0.093078,0.093078
temp,-0.090951,-0.143156,-0.122154,None,0.125559,0.112293,-0.048428,0.048428,-0.281744,0.281744
vomit_dur,-0.039075,-0.026711,-0.018737,0.125559,None,0.020258,-0.147209,0.147209,-0.086867,0.086867
cough_dur,-0.091015,-0.059380,-0.041915,0.112293,0.020258,None,-0.015304,0.015304,-0.108950,0.108950
diar_No,0.017397,0.053020,0.015691,-0.048428,-0.147209,-0.015304,None,-1.000000,0.040633,-0.040633
diar_Yes,-0.017397,-0.053020,-0.015691,0.048428,0.147209,0.015304,-1.000000,None,-0.040633,0.040633
head_No,0.069809,-0.108343,-0.093078,-0.281744,-0.086867,-0.108950,0.040633,-0.040633,None,-1.000000
head_Yes,-0.069809,0.108343,0.093078,0.281744,0.086867,0.108950,-0.040633,0.040633,-1.000000,None


'tGAN 4 SRA'

dday         0.611111
height       0.694444
weight       0.694444
temp         0.527778
vomit_dur    0.555556
cough_dur    0.444444
diar_No      0.583333
diar_Yes     0.583333
head_No      0.694444
head_Yes     0.694444
average      0.608333
dtype: float64

'tGAN 4 MSE table'

,dday,height,weight,temp,vomit_dur,cough_dur,diar_No,diar_Yes,head_No,head_Yes
dday,0.000000,0.301524,0.318359,0.375500,0.129292,0.037557,0.107354,0.122683,5.308513e-02,5.276953e-02
height,0.301524,0.000000,0.000885,0.008907,0.003731,0.245051,0.062906,0.070354,2.342499e-04,2.308390e-04
weight,0.318359,0.000885,0.000000,0.027936,0.000718,0.075120,0.028124,0.033429,5.649137e-04,5.777484e-04
temp,0.375500,0.008907,0.027936,0.000000,0.130447,0.002854,0.099466,0.123930,3.811595e-02,3.782954e-02
vomit_dur,0.129292,0.003731,0.000718,0.130447,0.000000,0.176366,0.000022,0.000013,6.967063e-01,6.960018e-01
cough_dur,0.037557,0.245051,0.075120,0.002854,0.176366,0.000000,0.079986,0.081052,8.605441e-02,8.635610e-02
diar_No,0.107354,0.062906,0.028124,0.099466,0.000022,0.079986,0.000000,0.000007,6.263326e-03,6.221130e-03
diar_Yes,0.122683,0.070354,0.033429,0.123930,0.000013,0.081052,0.000007,0.000000,8.020146e-03,7.966082e-03
head_No,0.053085,0.000234,0.000565,0.038116,0.696706,0.086054,0.006263,0.008020,0.000000e+00,6.311835e-11
head_Yes,0.052770,0.000231,0.000578,0.037830,0.696002,0.086356,0.006221,0.007966,6.311835e-11,0.000000e+00


In [7]:
display('MSE values of r for each synthetic data',MSE_series)

'MSE values of r for each synthetic data'

DoppelGANger_0814    0.003839
DoppelGANger_0824    0.048782
DoppelGANger_0827    0.112824
tGAN                 0.315190
tGAN 1               0.129868
tGAN 2               0.111499
tGAN 4               0.098235
dtype: float64

## Conclusion
* The lower the value of MSE, the smaller the average difference between the correlation coefficients between the real and synthetic data, hence the better the result.   

* DoppelGANger_0814 is good but it contains only 5 columns.  

* tGAN, gen 1 and gen 2 have large MSE values, by comparing the correlation tables we find that these generated datas have wronly too strong dependencies (r>0.5) between some columns.  

* DoppelGANger_0824 agrees with its original data in all r>0.5. DoppelGANger_0827 doesn't do well in categorical columns (diar and head), wrongly strong dependencies between these columns.

In [8]:
display('SRA for each column and synthetic data',sra_df)

'SRA for each column and synthetic data'

,DoppelGANger_0814,DoppelGANger_0824,DoppelGANger_0827,tGAN,tGAN 1,tGAN 2,tGAN 4
dday,1.0,0.361111,0.333333,0.694444,0.333333,0.861111,0.611111
weight,1.0,0.638889,0.444444,0.555556,0.527778,0.805556,0.694444
height,1.0,0.750000,0.555556,0.638889,0.527778,0.694444,0.694444
age,1.0,NaN,NaN,NaN,NaN,NaN,NaN
temp,1.0,0.444444,0.527778,0.750000,0.861111,0.361111,0.527778
average,1.0,0.602778,0.508333,0.600000,0.588889,0.666667,0.608333
vomit_dur,NaN,0.555556,0.583333,0.166667,0.611111,0.750000,0.555556
cough_dur,NaN,0.833333,0.472222,0.583333,0.500000,0.500000,0.444444
diar_No,NaN,0.583333,0.527778,0.500000,0.527778,0.833333,0.583333
diar_Yes,NaN,0.611111,0.500000,0.500000,0.555556,0.805556,0.583333


## Conclusion
* we can conclude that DoppleGANger_0814preserves the dependency ranking between columns very well.   

* By comparing the 'average', tGAN 2 is best at preserve the ranking. Though in the previous section we find that tGAN 2 tends to have a large r value in average.  
* DoppelGANger_0827 achives the lowest SRA by comparing the average.

## Possible Improvements in this method
Note in the TGAN data, it contains categorical columns e.g. diar_No and diar_Yes between which the _r_ is -1. This corresponding to the logic fact that if diar_No = 1, then diar_Yes = 0; if diar_No = 0, then diar_Yes = 1. A reasonable synthetic data has to respect this kind of 'logic' relationship.  

As a result, it's reasonable to say that that larger the absolute value of _r_ is, the more important the relationship is, that's why we choose to use MSE rather than MAE in quantitative evaluation, a possible improvement is adding weight according to _r_ rather than averaging the MSE.

**Reference:**
<!--[Text](link)-->
* [James Jordon, Jinsung Yoon, Mihaela van der Schaar. PATE-GAN: GENERATING SYNTHETIC DATA WITH
DIFFERENTIAL PRIVACY GUARANTEES](https://openreview.net/pdf?id=S1zk9iRqF7)